In [194]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
#Import the API key.
from config import g_key

In [195]:
city_data_df = pd.read_csv("weather_data/cities.csv")

In [196]:
#city_data_df.dtypes

In [197]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [198]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center = (30.0,31.0), zoom_level = 1.5)
# Assign the heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig 


Figure(layout=FigureLayout(height='420px'))

In [199]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [200]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations,weights=clouds,
dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.

fig

Figure(layout=FigureLayout(height='420px'))

In [201]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [202]:
preffered_cities = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp ) & (city_data_df["Max Temp"] <= max_temp)]
preffered_cities.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Sinnamary,GF,2022-06-15 12:26:45,5.3833,-52.9500,80.87,76,42,10.25
4,4,Bengkulu,ID,2022-06-15 12:25:32,-3.8004,102.2655,78.96,81,88,3.33
7,7,Male,MV,2022-06-15 12:26:48,4.1748,73.5089,85.95,79,75,11.50
11,11,Athmallik,IN,2022-06-15 12:26:50,20.7219,84.5348,88.93,58,95,10.85
15,15,Cotonou,BJ,2022-06-15 12:26:24,6.3654,2.4183,75.18,100,75,11.50
18,18,Victoria,HK,2022-06-15 12:26:04,22.2855,114.1577,84.38,86,96,5.99
20,20,Samarai,PG,2022-06-15 12:26:56,-10.6167,150.6667,79.41,84,100,10.60
28,28,Basco,PH,2022-06-15 12:27:00,20.4487,121.9702,84.67,72,100,1.36
29,29,Faanui,PF,2022-06-15 12:25:19,-16.4833,-151.7500,80.13,70,66,21.56
30,30,Karratha,AU,2022-06-15 12:27:01,-20.7377,116.8463,75.47,66,17,7.14


In [203]:
preffered_cities.count()


City_ID       191
City          191
Country       190
Date          191
Lat           191
Lng           191
Max Temp      191
Humidity      191
Cloudiness    191
Wind Speed    191
dtype: int64

In [204]:
preffered_cities = preffered_cities.dropna()

In [205]:
preffered_cities.count()

City_ID       190
City          190
Country       190
Date          190
Lat           190
Lng           190
Max Temp      190
Humidity      190
Cloudiness    190
Wind Speed    190
dtype: int64

In [206]:
hotel_df  = preffered_cities[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

In [207]:
hotel_df["Hotel Name"] = ""

In [208]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Sinnamary,GF,80.87,5.3833,-52.9500,
4,Bengkulu,ID,78.96,-3.8004,102.2655,
7,Male,MV,85.95,4.1748,73.5089,
11,Athmallik,IN,88.93,20.7219,84.5348,
15,Cotonou,BJ,75.18,6.3654,2.4183,
...,...,...,...,...,...,...
544,Andzhiyevskiy,RU,77.07,44.2300,43.0800,
545,Sola,VU,77.67,-13.8833,167.5500,
546,Mangai,CD,88.70,-4.0500,19.5333,
547,Maraa,BR,75.31,-1.8333,-65.3667,


In [209]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [210]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print1 = hotels["results"][0]["name"]
        print(f"Hotel Found : {print1}")
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel Found : ALETHEIA Appartement Sinnamary
Hotel Found : Grage Hotel Bengkulu
Hotel Found : Hotel Octave Maldives
Hotel Found : Mishra resident and Advocate chamber,Bangalasahi,Athamallik,Angul
Hotel Found : Novotel Cotonou Orisha
Hotel Found : Mini Central
Hotel Found : Nuli Sapi
Hotel Found : Nathaniel's Lodge
Hotel Found : Bora-Bora Pearl Beach Resort & Spa
Hotel Found : ibis Styles Karratha
Hotel Found : castelo pizzaria
Hotel Found : Hampton Inn Columbia-Downtown Historic District
Hotel Found : Pin Drop Hotel
Hotel Found : Sheraton Kauai Coconut Beach Resort
Hotel not found... skipping.
Hotel Found : HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel
Hotel Found : janelas do mar
Hotel Found : املاك خالد الشعشوع
Hotel Found : La Caz Oceane
Hotel Found : Hotel do Campo
Hotel Found : Scoop Guest House
Hotel Found : Uruna Bay Retreat
Hotel Found : Hotel Santika Palu
Hotel Found : Kiat Ngong Wetland BUNGALOW
Hotel Found : HOTEL QUINTA ANTIGUA
Hotel Found : Haji laloo memon house
Hotel Found : 

In [211]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Sinnamary,GF,80.87,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
4,Bengkulu,ID,78.96,-3.8004,102.2655,Grage Hotel Bengkulu
7,Male,MV,85.95,4.1748,73.5089,Hotel Octave Maldives
11,Athmallik,IN,88.93,20.7219,84.5348,"Mishra resident and Advocate chamber,Bangalasa..."
15,Cotonou,BJ,75.18,6.3654,2.4183,Novotel Cotonou Orisha


In [212]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [213]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [214]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))